In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
import numpy as np
import pandas as pd
import PreClf, MultiPreClf
from sklearn.datasets import fetch_openml
from sklearn.datasets import make_moons

import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# MLP from sklearn
from sklearn.neural_network import MLPClassifier


# MNIST 
X_, y_ = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas")
# Preprocessing normalisation 
X_ = X_ / 255

y_ = y_.astype(int)

# On-hot encoding
encoder = OneHotEncoder(sparse=False)
y_one_hot = encoder.fit_transform(y_.reshape(-1, 1))


X_train, X_test, y_train, y_test = train_test_split(X_, y_one_hot, test_size=0.2, random_state=42)


sk_mlp = MLPClassifier(hidden_layer_sizes=(100, 100), activation='relu', solver='adam', alpha=0.0001, batch_size='auto',
                          learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True,
                            random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
                                nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9,
                                    beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)


m:\ML\ML_regs\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [45]:
# # Train 
# sk_mlp.fit(X_train, y_train)

# # Predict
# y_pred = sk_mlp.predict(X_test)




Accuracy:  9.9585


In [63]:
model = MultiPreClf.MultilayerPerceptron(alpha=0.01, 
                                         n_iterations=2, 
                                         input_layer_size=784, 
                                         hidden_layers_size=[32, 16], 
                                         n_outputs=10, 
                                         regularization="l2", 
                                         lambda_reg=0.1)

model_two = MultiPreClf.MLP(alpha=0.01, n_iterations=5, 
                            input_layer_size=784, hidden_layers_size=[128, 64], 
                            n_outputs=10,
                            regularization="l2", 
                            lambda_reg=0.1)

In [60]:
# from sklearn.datasets import make_moons
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Create a simple dataset
# X, y = make_moons(n_samples=100, noise=0.2, random_state=42)

# # Reshape y for compatibility with the neural network
# y = y.reshape(-1, 1)

# # Split into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Initialize and train the Multilayer Perceptron
# mlp = MultiPreClf.MultilayerPerceptron(alpha=0.01, n_iterations=100, input_layer_size=2, hidden_layers_size=[10], n_outputs=1)
# training_loss, validation_loss = mlp.train(X_train, y_train)

# # Predict on the test set
# y_pred = mlp.predict(X_test)

# # Calculate the accuracy
# accuracy = accuracy_score(y_test, y_pred)
# accuracy



Iteration: 0
Epoch: 0, Training loss : 1.8327966395874447
Iteration: 1
Epoch: 1, Training loss : 7.894577455979586
Iteration: 2
Epoch: 2, Training loss : 7.894577455979586
Iteration: 3
Epoch: 3, Training loss : 7.894577455979586
Iteration: 4
Epoch: 4, Training loss : 7.894577455979586
Iteration: 5
Epoch: 5, Training loss : 7.894577455979586
Iteration: 6
Epoch: 6, Training loss : 7.894577455979586
Iteration: 7
Epoch: 7, Training loss : 7.894577455979586
Iteration: 8
Epoch: 8, Training loss : 7.894577455979586
Iteration: 9
Epoch: 9, Training loss : 7.894577455979586
Iteration: 10
Epoch: 10, Training loss : 7.894577455979586
Iteration: 11
Epoch: 11, Training loss : 7.894577455979586
Iteration: 12
Epoch: 12, Training loss : 7.894577455979586
Iteration: 13
Epoch: 13, Training loss : 7.894577455979586
Iteration: 14
Epoch: 14, Training loss : 7.894577455979586
Iteration: 15
Epoch: 15, Training loss : 7.894577455979586
Iteration: 16
Epoch: 16, Training loss : 7.894577455979586
Iteration: 17
Ep

m:\ML\ML_regs\Preceptron\MultiPreClf.py:221: RuntimeWarning: overflow encountered in exp
  # load model weights


0.3333333333333333

In [53]:
for layer in range(len(model.weights) - 1, -1, -1):
    print(layer)
    print(model.weights[layer].shape)

2
(16, 10)
1
(32, 16)
0
(784, 32)


In [54]:
for layer in range(1, len(model.weights) + 1):
    print(layer)

1
2
3


In [ ]:
training_loss, validation_loss = model_two.train(X_train, y_train, X_test, y_test)


In [34]:
# Step 3: Evaluate the Model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Step 4: Calculate Accuracy
accuracy = model.accuracy(y_test_classes, y_pred_classes)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 11.43%


In [37]:
y_pred

array([[0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       ...,
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438],
       [0.0984442 , 0.11360162, 0.09985351, ..., 0.10101594, 0.09880851,
        0.09915438]])

In [35]:
y_pred_classes

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [36]:
y_test_classes

array([8, 4, 8, ..., 2, 7, 1], dtype=int64)

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [6]:
train = pd.read_csv("C:/Users/denis/Downloads/digit-recognizer/train.csv")
test = pd.read_csv("C:/Users/denis/Downloads/digit-recognizer/test.csv")

In [9]:
data = np.array(train)

m , n = data.shape

In [10]:
np.random.shuffle(data)

In [11]:
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]

In [15]:
def  init_params():

    W_one = np.random.rand(10, 784) - 0.5
    b_one = np.random.rand(10, 1) - 0.5

    W_two = np.random.rand(10, 10) - 0.5
    b_two = np.random.rand(10, 1) - 0.5

    return W_one, b_one, W_two, b_two

def relu(Z):

    return np.maximum(0, Z)

def softmax(Z):

    A = np.exp(Z) / sum(np.exp(Z))

    return A

def forward_prop(W_one, b_one, W_two, b_two, X):

    Z_one = W_one.dot(X) + b_one
    A_one = relu(Z_one)

    Z_two = W_two.dot(A_one) + b_two
    A_two = softmax(Z_two)

    return Z_one, A_one, Z_two, A_two